# Constance de Salm #

Extracting and generating Tweets for CdS. Twtteraccound: https://twitter.com/constancesalm

Importing the needed libraries.

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns

Loading the Dataset via pandas.

In [2]:
df = pd.read_csv("lib/20211116_Constance_de_Salm_Korrespondenz_Inventar_Briefe.csv", sep=';', encoding='utf8')
print(list(df))

['FuD-Key', 'letzte Änderung', 'Dokumenttyp', 'URL', 'Nr.', 'Verfasser', 'GND (Verfasser)', 'VIAF (Verfasser)', ' Empfänger', 'GND (Empfänger)', 'VIAF (Empfänger)', 'Datierung (JJJJ-MM-TT)', 'Alternative Datierung', 'Ausstellungsort', 'Geonames (Ausstellungsort)', 'Empfangsort', 'Geonames (Empfangsort)', 'Vorlage', 'Vollständigkeit des Archivals', 'Erläuterung Unvollständigkeit', 'Umfang in Seiten', 'Anfang des Briefes', 'Inhaltliche Stichpunkte', 'Weitere Personen', 'GND (Weitere Personen)', 'VIAF (Weitere Personen)', 'Orte', 'Geonames (Orte)', 'Schlagwörter', 'Editionen', 'Vermerk', 'Bestand', 'Verweise', 'Digitalisate: Dateiname (Signatur)', 'Zitierempfehlung']


Extracting just the letters which were by Constance de Salm herself.

In [3]:

df_cds = pd.DataFrame(columns=list(df))
for i, v in df.iterrows():
    if type(v["Verfasser"]) is not str: continue
    if re.search(r'.*Constance.*', v["Verfasser"]):
        df_cds = df_cds.append(v, ignore_index=True)

## Generating the tweets ##
How many Tweets can be used for the Twitter-Account (based on the length, which is limited by Twitter [280])


In [4]:
length = []
long_tweets = []
for i, v in df_cds.iterrows():
    tweet = '"{0}..." à {1}, {2} ({3})'.format(v["Anfang des Briefes"],
                                            v[" Empfänger"],
                                            v["Datierung (JJJJ-MM-TT)"],
                                            v["URL"]).replace('\n', '').replace('\t', '')
    length.append(len(tweet))
    if len(tweet) > 280:
        long_tweets.append(tweet)
print("{} Tweets are shorter then 280 chars.".format(len(list(filter(lambda x: x<=280, length)))))

2764 Tweets are shorter then 280 chars.


In [5]:
import numpy as np

def generate_next_day(d, m, y):
    months = {1: 31, 2:28, 3: 31, 4: 30, 5: 31, 6: 30, 7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    d = d + 1
    if d > months[m]:
        d = 1
        m +=1
        if m > 12:
            m = 1
            y += 1
    return y, m, d

tweets_df = pd.DataFrame(columns=["Date", "Time", "Tweet", "Picture", "Longitude", "Lattitude"])
geo_data = pd.read_csv("lib/CdS_Places.csv", encoding="utf8", index_col='id')

day = (2021, 11, 26)
change = 0
for i, v in df_cds.iterrows():
    row = {}
    tweet = '"{0}" à {1}, {2} ({3})'.format(str(v["Anfang des Briefes"]),
                                            v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.',
                                            v["Datierung (JJJJ-MM-TT)"],
                                            v["URL"])
    if len(tweet) > 280: continue
    row["Tweet"] = tweet
    row["Date"] = "{}.{}.{}".format(day[2], day[1], day[0])
    time_tweet = ""
    h = np.random.randint(7, 21)
    m = np.random.randint(0, 59)
    s = np.random.randint(0, 59)
    h = str(h) if len(str(h)) == 2 else "0" + str(h)
    m = str(m) if len(str(m)) == 2 else "0" + str(m)
    s = str(s) if len(str(s)) == 2 else "0" + str(s)
    row["Time"] = "{}:{}:{}".format(h, m, s)
    geo_information = geo_data.loc[geo_data.url == v['Geonames (Ausstellungsort)']]
    try:
        row["Longitude"] = geo_information.iloc[0]['long']
        row["Lattitude"] = geo_information.iloc[0]['lat']
    except IndexError:
        True
    if change == 1:
        day = generate_next_day(day[2], day[1], day[0])
        change = 0
    else:
        change += 1
    tweets_df = tweets_df.append(row, ignore_index=True)

In [6]:
tweets_df

,Date,Time,Tweet,Picture,Longitude,Lattitude
0,26.11.2021,13:48:58,"""Cher et excellent voisin, j'ai remis de jour ...",NaN,6.50000,51.10000
1,26.11.2021,19:23:51,"""Je vais retourner à Paris dans 15 jours, Mons...",NaN,6.50000,51.10000
2,27.11.2021,13:18:29,"""Nous avons bien regretté, Monsieur, de ne pas...",NaN,6.50000,51.10000
3,27.11.2021,08:04:42,"""Je ne vous ai pas encore écrit, Monsieur,"" à ...",NaN,6.50000,51.10000
4,28.11.2021,12:31:00,"""Je ne serai jamais embarrassée, Monsieur, qua...",NaN,NaN,NaN
...,...,...,...,...,...,...
2804,29.9.2025,18:33:34,"""Je n’entends point parler de vous, Monsieur, ...",NaN,6.50000,51.10000
2805,29.9.2025,19:17:15,"""Vous aurez peut-être été surpris, Monsieur, d...",NaN,6.08342,50.77664
2806,30.9.2025,07:30:31,"""Je reçois à l'instant votre lettre de la camp...",NaN,6.50000,51.10000
2807,30.9.2025,14:06:00,"""J’ai bien tardé à répondre à votre dernière l...",NaN,NaN,NaN


In [7]:
tweets_df.to_excel("lib/Lettre_de_CdS.xlsx", encoding="utf8")
tweets_df.to_csv("lib/Lettre_de_CdS.tsv", encoding="utf8", sep="\t")

## Creating Flashcards
We start now with the Creation of the Flashcards based on the script found in the directory "flash_cards".
The cards will be saved into the directory: "flash_cards_png".

In [8]:
from flash_cards.convert_svg import *

file = open('flash_cards/flashcard_cds.svg', encoding='utf8')
svg_format = file.read()
file.close()

for i, v in df_cds.iterrows():
    row = {}
    text = str(v["Anfang des Briefes"]).replace('\n', '').replace('\t', '').replace('  ', ' ').strip() + ' ...'
    flash_card_text = create_flashcard_text(text)
    flash_card = svg_format.format(text=flash_card_text,
                                   date=v["Datierung (JJJJ-MM-TT)"],
                                   name=v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.')
    with open('flash_cards/CdS_Test.svg', 'w', encoding='utf8') as f:
        f.write(flash_card)

    row["Tweet"] = v["URL"]
    row["Date"] = "{}.{}.{}".format(day[2], day[1], day[0])
    time_tweet = ""
    h = np.random.randint(7, 21)
    m = np.random.randint(0, 59)
    s = np.random.randint(0, 59)
    h = str(h) if len(str(h)) == 2 else "0" + str(h)
    m = str(m) if len(str(m)) == 2 else "0" + str(m)
    s = str(s) if len(str(s)) == 2 else "0" + str(s)
    row["Time"] = "{}:{}:{}".format(h, m, s)
    geo_information = geo_data.loc[geo_data.url == v['Geonames (Ausstellungsort)']]
    try:
        row["Longitude"] = geo_information.iloc[0]['long']
        row["Lattitude"] = geo_information.iloc[0]['lat']
    except IndexError:
        True
    if change == 1:
        day = generate_next_day(day[2], day[1], day[0])
        change = 0
    else:
        change += 1
    tweets_df = tweets_df.append(row, ignore_index=True)
    convert_svg('flash_cards/CdS_Test.svg', 'flash_cards/png/{}.png'.format(v["FuD-Key"]))
    break

			<tspan class="TextPosition" x="3150"> Voulez-vous bien, Monsieur, me</tspan>
			<tspan class="TextPosition" x="3150" dy="1.2em">rendre un petit service: je vais vous</tspan>
			<tspan class="TextPosition" x="3150" dy="1.2em">expliquer d'abord, et en détail, de</tspan>
			<tspan class="TextPosition" x="3150" dy="1.2em">quoi il s'agit, afin de n'avoir plus à y</tspan>
			<tspan class="TextPosition" x="3150" dy="1.2em">penser et de pouvoir répondre</tspan>
			<tspan class="TextPosition" x="3150" dy="1.2em">tranquillement à votre lettre. ...</tspan>

('FrutigerNextCondensed', True)


Warn: Can't find .pfb for face 'Times-Roman'


RenderPMError: Can't setFont(Times-Roman) missing the T1 files?
Originally <class 'TypeError'>: makeT1Font() argument 2 must be str, not None

## Visualizing the Informations ##

In [ ]:
#!pip install geopandas
#!pip install geoplot
#import geopandas as gp
#import geoplot as gplt
#from shapely.geometry import Point
import folium

In [ ]:
m = folium.Map()
geo_data = pd.read_csv("lib/CdS_Places.csv", encoding="utf8", index_col='id')

for i, v in df_cds.iterrows():
    geo_information = geo_data.loc[geo_data.url == v['Geonames (Ausstellungsort)']]
    try:
        lng = geo_information.iloc[0]['long']
        lat = geo_information.iloc[0]['lat']
        folium.Marker([lat, lng],
                      popup="<i>à {}, le </i>".format(v["Anfang des Briefes"]),
                      tooltip="à {}, le ".format(v[" Empfänger"].split(', ')[0] if type(v[" Empfänger"]) is str else 'N.N.', v["Datierung (JJJJ-MM-TT)"])).add_to(m)
    except IndexError:
        True